In [1]:
from torch import nn


class SentenceClassifier(nn.Module):
    # 입력층 : Embedding Layer
    # 은닉층 : RNN/LSTM Layer
    # 은닉층 : Dropout Layer
    # 출력층 : Linear Layer
    def __init__(
        self,
        n_vocab,  # 단어 개수
        hidden_dim, # 은닉 층 개수
        embedding_dim, # 임베딩할 차원 크기
        n_layers, 
        dropout=0.5,
        bidirectional=True,
        model_type="lstm"
    ):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,     # 첫번째 인자 : 단어 총 개수
            embedding_dim=embedding_dim, # 두번째 인자 : 변환할 차원 크기
            padding_idx=0
        )
        if model_type == "rnn":
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1) # 양방향이면, 층이 2개니까 출력값도 2개가 나오기 때문 
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings) # 출력이 두개라서, _을 통해 unpacking
        last_output = output[:, -1, :]  # 마지막 시점의 결괏값만 분리함
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [5]:
import pandas as pd
from Korpora import Korpora


corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\mathn\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\mat

In [6]:
train = corpus_df.sample(frac=0.9, random_state=42) 
test = corpus_df.drop(train.index) # 해당하는 인덱스 드랍

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

|       | text                                                                                     |   label |
|------:|:-----------------------------------------------------------------------------------------|--------:|
| 33553 | 모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. |       1 |
|  9427 | 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...                                    |       0 |
|   199 | 신날 것 없는 애니.                                                                       |       0 |
| 12447 | 잔잔 격동                                                                                |       1 |
| 39489 | 오랜만에 찾은 주말의 명화의 보석                                                         |       1 |
Training Data Size : 45000
Testing Data Size : 5000


In [7]:
from konlpy.tag import Okt
from collections import Counter

# 데이터 토큰화 및 단어 사전 구축 
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens) # update 메서드를 사용하여 갱신 
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab): # most_common을 사용하여 빈도수가 높은 순으로 정렬
        vocab.append(token) # 단어들 추가
    return vocab 


tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text] # 형태소 분석기를 사용하여 토큰화
test_tokens = [tokenizer.morphs(review) for review in test.text]

vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"]) # 패딩을 위한 토큰, 모르는 단어를 위한 토큰 추가
token_to_id = {token: idx for idx, token in enumerate(vocab)} # dict_comprehension을 주어 딕셔너리 생성
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002


In [8]:
import numpy as np

# 정수 인코딩 및 패딩 
def pad_sequences(sequences, max_length, pad_value):
    result = list()
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)


unk_id = token_to_id["<unk>"]
train_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length = 32
pad_id = token_to_id["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[ 223 1716   10 4036 2095  193  755    4    2 2330 1031  220   26   13
 4839    1    1    1    2    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[3307    5 1997  456    8    1 1013 3906    5    1    1   13  223   51
    3    1 4684    6    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [9]:
import torch
from torch.utils.data import TensorDataset, DataLoader


train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [10]:
import torch
from torch.utils.data import TensorDataset, DataLoader


train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

C:\Users\mathn\AppData\Local\Temp\ipykernel_19372\2284756583.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_ids = torch.tensor(train_ids)
C:\Users\mathn\AppData\Local\Temp\ipykernel_19372\2284756583.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_ids = torch.tensor(test_ids)


In [11]:
from torch import optim


n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [12]:
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")


def test(model, datasets, criterion, device):
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")


epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6870641708374023
Train Loss 500 : 0.6933769105437273
Train Loss 1000 : 0.688477501049861
Train Loss 1500 : 0.6726846184673347
Train Loss 2000 : 0.6543901640435924
Train Loss 2500 : 0.6345096552004961
Val Loss : 0.4956704857059942, Val Accuracy : 0.762
Train Loss 0 : 0.4850248098373413
Train Loss 500 : 0.4762774507799548
Train Loss 1000 : 0.47200047992147526
Train Loss 1500 : 0.46481517081376633
Train Loss 2000 : 0.4584861027262081
Train Loss 2500 : 0.45470732140665004
Val Loss : 0.4318852040428704, Val Accuracy : 0.7996
Train Loss 0 : 0.5159556865692139
Train Loss 500 : 0.3817289995992493
Train Loss 1000 : 0.3810801488774402
Train Loss 1500 : 0.38281998270595174
Train Loss 2000 : 0.38153420540986804
Train Loss 2500 : 0.3807856669840885
Val Loss : 0.40622627035307046, Val Accuracy : 0.8064
Train Loss 0 : 0.14860151708126068
Train Loss 500 : 0.33334633784587986
Train Loss 1000 : 0.334805947094858
Train Loss 1500 : 0.3373573270466906
Train Loss 2000 : 0.33858437465130536


In [13]:
token_to_embedding = dict()
embedding_matrix = classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix):
    token_to_embedding[word] = emb

token = vocab[1000]
print(token, token_to_embedding[token])

보고싶다 [-0.29427832 -0.44090214 -0.68614966 -0.17272356  0.06190613 -1.3241094
  0.19906461 -0.01356532 -0.39992237 -0.6586416  -0.2405167  -0.35442513
 -1.9691424   1.0417427   1.0318938  -0.81515056  2.56317     1.7430211
  0.10699182 -0.14589234  1.739308    1.352874   -0.08705691  1.6088581
 -1.5788411   1.0749993   0.32469875  0.30129203 -0.25626016  0.6967822
 -2.2814586   0.14408009  1.0303414   1.06877    -0.3929306  -0.19429104
  2.2218964   1.0684528  -1.1541137  -0.02025843  2.4053917  -1.0499542
  0.5600085   1.167036   -0.5839037  -0.1914809   0.6941734  -0.41712353
 -0.7096074  -0.24548636  0.69249105  0.47792694 -0.39372078 -1.0353252
 -1.6841859  -0.99337053  0.11846524 -0.39688146  1.3968252   1.0624415
 -1.6396984  -0.9159098   1.5313836  -0.58152     0.92995155  0.19711426
  0.6550636   0.2944267   0.46837565 -0.37530637 -1.4565248   0.3184026
  0.37482938 -0.27328184 -1.1594085   0.38577455 -0.86808693  1.1142882
  0.10117419 -0.17475167 -0.6563677  -0.98998624 -0.127